load libraries

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

In [23]:
# dir = "/ext/Data/distracted_driver_detection/"
dir = "F:\BBBBBBBBBBBBBBBBB\Datas"

model_image_size = (240, 360)
fine_tune_layer = 152
# final_layer = 176
final_layer = 177
    # BatchNormalization 和Dropout 通常不包含权重，无法通过 .get_weights() 来访问它们；
    # 如果你试图提取权重，请确保访问的是包含可训练参数的层，例如卷积层（Conv2D）或全连接层（Dense）。
visual_layer = 172
batch_size = 128

def lambda_func(x):
    x /= 255.
    x -= 0.5
    x *= 2
    return x

# 加载数据集

load train data

In [3]:
train_gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)
gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
)
train_generator = train_gen.flow_from_directory(os.path.join(dir, 'train'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to train type {}".format(train_generator.class_indices))
valid_generator = gen.flow_from_directory(os.path.join(dir, 'valid'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to valid type {}".format(valid_generator.class_indices))

C:\Users\13904\anaconda3\lib\site-packages\keras\preprocessing\image.py:1460: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn(


Found 20787 images belonging to 10 classes.
subdior to train type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}
Found 1637 images belonging to 10 classes.
subdior to valid type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


# 构建模型

https://github.com/fchollet/keras/blob/master/keras/applications/resnet50.py


In [4]:
input_tensor = Input((*model_image_size, 3))
x = input_tensor
# if lambda_func:
#     x = Lambda(lambda_func)(x)

base_model = ResNet50(input_tensor=Input((*model_image_size, 3)), weights='imagenet', include_top=False)

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(base_model.input, x)

print("total layer count {}".format(len(base_model.layers)))

for i in range(fine_tune_layer):
    model.layers[i].trainable = False



94765736/94765736 [==============================] - 9s 0us/step
total layer count 175


# 训练模型

In [5]:
print("train_generator.samples = {}".format(train_generator.samples))
print("valid_generator.samples = {}".format(valid_generator.samples))
steps_train_sample = train_generator.samples // 128 + 1
steps_valid_sample = valid_generator.samples // 128 + 1

train_generator.samples = 20787
valid_generator.samples = 1637


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=6, validation_data=valid_generator, validation_steps=steps_valid_sample)

model.save("models/resnet50-imagenet-finetune{}-adam.h5".format(fine_tune_layer))
print("model saved!")

C:\Users\13904\AppData\Local\Temp\ipykernel_11072\1649610113.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=6, validation_data=valid_generator, validation_steps=steps_valid_sample)
C:\Users\13904\anaconda3\lib\site-packages\keras\preprocessing\image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
C:\Users\13904\anaconda3\lib\site-packages\keras\preprocessing\image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/6
163/163 [==============================] - 1527s 9s/step - loss: 0.2107 - accuracy: 0.9339 - val_loss: 1.6385 - val_accuracy: 0.7025
Epoch 2/6
163/163 [==============================] - 1522s 9s/step - loss: 0.0475 - accuracy: 0.9851 - val_loss: 1.3144 - val_accuracy: 0.7318
Epoch 3/6
163/163 [==============================] - 1517s 9s/step - loss: 0.0333 - accuracy: 0.9903 - val_loss: 2.2476 - val_accuracy: 0.6213
Epoch 4/6
163/163 [==============================] - 1518s 9s/step - loss: 0.0217 - accuracy: 0.9935 - val_loss: 1.2544 - val_accuracy: 0.6799
Epoch 5/6
163/163 [==============================] - 1517s 9s/step - loss: 0.0228 - accuracy: 0.9930 - val_loss: 1.4347 - val_accuracy: 0.7239
Epoch 6/6
163/163 [==============================] - 1519s 9s/step - loss: 0.0139 - accuracy: 0.9959 - val_loss: 2.0465 - val_accuracy: 0.6420
model saved!


In [7]:
model.compile(optimizer=RMSprop(lr=1*0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=6, validation_data=valid_generator, validation_steps=steps_valid_sample)

model.save("models/resnet50-imagenet-finetune{}.h5".format(fine_tune_layer))
print("model saved!")

C:\Users\13904\anaconda3\lib\site-packages\keras\optimizers\legacy\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\13904\AppData\Local\Temp\ipykernel_11072\365326434.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=6, validation_data=valid_generator, validation_steps=steps_valid_sample)


Epoch 1/6
163/163 [==============================] - 1517s 9s/step - loss: 0.0109 - accuracy: 0.9968 - val_loss: 1.3901 - val_accuracy: 0.6988
Epoch 2/6
163/163 [==============================] - 1522s 9s/step - loss: 0.0064 - accuracy: 0.9983 - val_loss: 1.3989 - val_accuracy: 0.6982
Epoch 3/6
163/163 [==============================] - 1520s 9s/step - loss: 0.0045 - accuracy: 0.9987 - val_loss: 1.4210 - val_accuracy: 0.7013
Epoch 4/6
163/163 [==============================] - 1522s 9s/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 1.4310 - val_accuracy: 0.7037
Epoch 5/6
163/163 [==============================] - 1518s 9s/step - loss: 0.0033 - accuracy: 0.9991 - val_loss: 1.5054 - val_accuracy: 0.7019
Epoch 6/6
163/163 [==============================] - 1518s 9s/step - loss: 0.0026 - accuracy: 0.9992 - val_loss: 1.5541 - val_accuracy: 0.7019
model saved!


# 可视化模型

https://keras.io/visualization/

In [8]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import *

model = load_model("models/resnet50-imagenet-finetune{}.h5".format(fine_tune_layer))
print("load successed")

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

load successed


# CAM 可视化

http://cnnlocalization.csail.mit.edu/

![](http://cnnlocalization.csail.mit.edu/framework.jpg)

$cam = (P-0.5)*w*output$

* cam: 类激活图 X\*X
* P: 概率
* output: 卷积层的输出 2048\*1
* w: 卷积核的权重 X\*X\*2048

In [14]:
z = zip([x.name for x in model.layers], range(len(model.layers)))
for k, v in z:
    print("{} - {}".format(k,v))

input_2 - 0
conv1_pad - 1
conv1_conv - 2
conv1_bn - 3
conv1_relu - 4
pool1_pad - 5
pool1_pool - 6
conv2_block1_1_conv - 7
conv2_block1_1_bn - 8
conv2_block1_1_relu - 9
conv2_block1_2_conv - 10
conv2_block1_2_bn - 11
conv2_block1_2_relu - 12
conv2_block1_0_conv - 13
conv2_block1_3_conv - 14
conv2_block1_0_bn - 15
conv2_block1_3_bn - 16
conv2_block1_add - 17
conv2_block1_out - 18
conv2_block2_1_conv - 19
conv2_block2_1_bn - 20
conv2_block2_1_relu - 21
conv2_block2_2_conv - 22
conv2_block2_2_bn - 23
conv2_block2_2_relu - 24
conv2_block2_3_conv - 25
conv2_block2_3_bn - 26
conv2_block2_add - 27
conv2_block2_out - 28
conv2_block3_1_conv - 29
conv2_block3_1_bn - 30
conv2_block3_1_relu - 31
conv2_block3_2_conv - 32
conv2_block3_2_bn - 33
conv2_block3_2_relu - 34
conv2_block3_3_conv - 35
conv2_block3_3_bn - 36
conv2_block3_add - 37
conv2_block3_out - 38
conv3_block1_1_conv - 39
conv3_block1_1_bn - 40
conv3_block1_1_relu - 41
conv3_block1_2_conv - 42
conv3_block1_2_bn - 43
conv3_block1_2_relu - 

http://docs.opencv.org/trunk/d3/d50/group__imgproc__colormap.html

![](http://docs.opencv.org/trunk/colorscale_jet.jpg)

In [24]:
import matplotlib.pyplot as plt
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def show_heatmap_image(model_show, weights_show):
    test_dir = os.path.join(dir,  "test" )
    image_files = glob.glob(os.path.join(test_dir,"*"))
    print(len(image_files))
    
    plt.figure(figsize=(12, 24))
    for i in range(10):
        plt.subplot(5, 2, i+1)
        img = cv2.imread(image_files[2000*i+113])
        img = cv2.resize(img,  (model_image_size[1],model_image_size[0]))
        x = img.copy()
        x.astype(np.float32)
        out, predictions = model_show.predict(np.expand_dims(x, axis=0))
        predictions = predictions[0]
        out = out[0]
        
        max_idx = np.argmax(predictions)
        prediction = predictions[max_idx]

        status = ["safe driving",  " texting - right",  "phone - right",  "texting - left",  "phone - left",  
                  "operation radio", "drinking", "reaching behind", "hair and makeup", "talking"]

        plt.title('c%d |%s| %.2f%%' % (max_idx , status[max_idx], prediction*100))
    
        cam = (prediction - 0.5) * np.matmul(out, weights_show)
        cam = cam[:,:,max_idx]
        cam -= cam.min()
        cam /= cam.max()
        cam -= 0.2
        cam /= 0.8

        cam = cv2.resize(cam, (model_image_size[1],model_image_size[0]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam <= 0.2)] = 0

        out = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)

        plt.axis('off')
        plt.imshow(out[:,:,::-1])
print("done")

done


In [25]:
print("Total layers in the model:", len(model.layers))
print("final_layer index:", final_layer)
if final_layer >= len(model.layers):
    print(f"Error: final_layer index {final_layer} is out of range. Model only has {len(model.layers)} layers.")
else:
    weights = model.layers[final_layer].get_weights()[0]


Total layers in the model: 178
final_layer index: 177


In [ ]:
weights = model.layers[final_layer].get_weights()[0]
layer_output = model.layers[visual_layer].output
model2 = Model(model.input, [layer_output, model.output])
print("layer_output {0}".format(layer_output))
print("weights shape {0}".format(weights.shape))
show_heatmap_image(model2, weights)

layer_output KerasTensor(type_spec=TensorSpec(shape=(None, 8, 12, 2048), dtype=tf.float32, name=None), name='conv5_block3_3_bn/FusedBatchNormV3:0', description="created by layer 'conv5_block3_3_bn'")
weights shape (2048, 10)
79726


In [ ]:
def gen_kaggle_csv(model,  model_image_size, csv_name):
    dir = "/ext/Data/distracted_driver_detection/"

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(dir + "test/",  model_image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
#     s = test_generator.__dict__
#     del s['filenames']
#     print(s)
    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    print("y_pred shape {}".format(y_pred.shape))
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(y_pred[:3])

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        l.append( [name, *y_pred[i]] )

    l = np.array(l)
    data = {'img': l[:,0]}
    for i in range(10):
        data["c%d"%i] = l[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df.head(10)
    df = df.sort_values(by='img')
    df.to_csv(csv_name, index=None, float_format='%.3f')
    print("csv saved")

print("done")

In [ ]:
gen_kaggle_csv(model,  model_image_size, 'csv/resnet50-imagenet-finetune{}-pred.csv'.format(fine_tune_layer))